Homework 3: Sentiment Analysis
----

The following instructions apply to all notebooks and `.py` files you submit for this homework.

Due date: April 15th, 2024 11:59 PM (EST)

Total Points: (105)
- Task 0: 05 points
- Task 1: 10 points
- Task 2: 20 points
- Task 3: 25 points
- Task 4: 40 points (question in LSTM_EncDec.ipynb)

Goals:
- understand the difficulties of counting and probabilities in NLP applications
- work with real world data using different approaches to classification
- stress test your model (to some extent)


Allowed python modules:
- `numpy`, `matplotlib`, `keras`, `pytorch`, `nltk`, `pandas`, `sci-kit learn` (`sklearn`), `seaborn`, and all built-in python libraries (e.g. `math` and `string`)
- if you would like to use a library not on this list, please check with us on Campuswire first.
- all *necessary* imports have been included for you (all imports that we used in our solution)

Instructions:
- Complete outlined problems in this notebook.
- When you have finished, __clear the kernel__ and __run__ your notebook "fresh" from top to bottom. Ensure that there are __no errors__.
    - If a problem asks for you to write code that does result in an error (as in, the answer to the problem is an error), leave the code in your notebook but commented out so that running from top to bottom does not result in any errors.
- Double check that you have completed Task 0.
- Submit your work on Gradescope.
- Double check that your submission on Gradescope looks like you believe it should.

Names & Sections
----
Names: 

Task 0: Name, References, Reflection (5 points)
---

References
---
List the resources you consulted to complete this homework here. Write one sentence per resource about what it provided to you. If you consulted no references to complete your assignment, write a brief sentence stating that this is the case and why it was the case for you.

- Python and PyTorch documentations.
- Lecture notes and slides for revising the concepts.
- Youtube for concept related content.

(Example)
- https://docs.python.org/3/tutorial/datastructures.html
    - Read about the the basics and syntax for data structures in python.

AI Collaboration
---
Following the *Policy on the use of Generative AI* in the syllabus, please cite any LLMs that you used here and briefly describe what you used them for, including to improve language clarity in the written sections.

Reflection
----
Answer the following questions __after__ you complete this assignment (no more than 1 sentence per question required, this section is graded on completion):

1. Does this work reflect your best effort? 
- Yes
2. What was/were the most challenging part(s) of the assignment? 
- Finetuning the LSTM and data preprocessing
3. If you want feedback, what function(s) or problem(s) would you like feedback on and why? 
- Feedback on how best to debug training issues and the solutions.
4. Briefly reflect on how your partnership functioned--who did which tasks, how was the workload on each of you individually as compared to the previous homeworks, etc. 
- Individual Assignment.

Task 1: Provided Data Write-Up (10 points)
---

Every time you use a data set in an NLP application (or in any software application), you should be able to answer a set of questions about that data. Answer these now. Default to no more than 1 sentence per question needed. If more explanation is necessary, do give it.

This is about the __provided__ movie review data set.

1. Where did you get the data from? The provided dataset(s) were sub-sampled from https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
2. (1 pt) How was the data collected (where did the people acquiring the data get it from and how)? 
3. (2 pts) How large is the dataset (answer for both the train and the dev set, separately)? (# reviews, # tokens in both the train and dev sets)
4. (1 pt) What is your data? (i.e. newswire, tweets, books, blogs, etc)
5. (1 pt) Who produced the data? (who were the authors of the text? Your answer might be a specific person or a particular group of people)
6. (2 pts) What is the distribution of labels in the data (answer for both the train and the dev set, separately)?
7. (2 pts) How large is the vocabulary (answer for both the train and the dev set, separately)?
8. (1 pt) How big is the overlap between the vocabulary for the train and dev set?

1. The provided dataset(s) were sub-sampled from https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews.
2. Data was collected from IMDb by restricting to a maximum of 30 reviews per movie to avoid correlated ratings and ensuring equal number of negative and positive reviews in the whole dataset.
3. Reviews: Train-1600, dev-200. Tokens: Train-425345, Dev-54599. (Tokenized using word_tokenize from nltk).
4. The data comprises user-submitted text movie reviews with the sentiment labels as positive/negative.
5. Authors of the text were the users who posted the reviews on IMDB, the dataset was put together by {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher} (Ref: https://ai.stanford.edu/~amaas/data/sentiment/)
6. Train: Positive-804, Negative-796. Dev: Positive-105, Negative-95.
7. Vocab size: Train-30705 , Dev-8953. (Based on nltk word_tokenize).
8. Overlap between train and dev vocab: 6574.

In [1]:
import pandas as pd

In [2]:
train_data_file = 'movie_reviews_train.txt'
train_df = pd.read_csv(train_data_file, sep='\t', header=None, names=['id', 'review', 'label'])[['review', 'label']]
X_train, y_train = train_df['review'].values, train_df['label'].values

dev_data_file = 'movie_reviews_dev.txt'
dev_df = pd.read_csv(dev_data_file, sep='\t', header=None, names=['id', 'review', 'label'])[['review', 'label']]
X_dev, y_dev = dev_df['review'].values, dev_df['label'].values

test_data_file = 'movie_reviews_test.txt'
test_df = pd.read_csv(test_data_file, sep='\t', header=None, names=['id', 'review', 'label'])[['review', 'label']]
X_test, y_test = test_df['review'].values, test_df['label'].values

In [3]:
train_df.info()
dev_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  1600 non-null   object
 1   label   1600 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 25.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  200 non-null    object
 1   label   200 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 3.3+ KB


In [4]:
from nltk.tokenize import word_tokenize
def num_tokens(data):
    vocab = set()
    num_tokens = 0
    for doc in data:
        tokens = word_tokenize(doc)
        num_tokens += len(tokens)
        vocab.update(tokens)
    return num_tokens, len(vocab), vocab

train_num_tokens, train_num_vocab, train_vocab = num_tokens(X_train)
dev_num_tokens, dev_num_vocab, dev_vocab = num_tokens(X_dev)
print('Number of tokens in train data:', train_num_tokens)
print('Number of tokens in dev data:', dev_num_tokens)
print('Number of vocab in train data:', train_num_vocab)
print('Number of vocab in dev data:', dev_num_vocab)

print('Overlap between train and dev vocab:', len(train_vocab & dev_vocab))

Number of tokens in train data: 425345
Number of tokens in dev data: 54599
Number of vocab in train data: 30705
Number of vocab in dev data: 8953
Overlap between train and dev vocab: 6574


In [5]:
print('Distribution of labels in train data:')
print(train_df['label'].value_counts())
print('Distribution of labels in dev data:')
print(dev_df['label'].value_counts())

Distribution of labels in train data:
label
1    804
0    796
Name: count, dtype: int64
Distribution of labels in dev data:
label
1    105
0     95
Name: count, dtype: int64


Task 2: Train a Logistic Regression Model (20 points)
----
1. Implement a custom function to read in a dataset, and return a list of tuples, using the Tf-Idf feature extraction technique.
2. Compare your implementation to `sklearn`'s TfidfVectorizer (imported below) by timing both on the provided datasets using the time module.
3. Using each set of features, and `sklearn`'s implementation of `LogisticRegression`, train a machine learning model to predict sentiment on the given dataset.

In [6]:
import nltk
#nltk.download('punkt')
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from collections import Counter, defaultdict
import time
from nltk.corpus import stopwords

#nltk.download('stopwords')
stopwords = stopwords.words('english')

In [7]:
# The following function reads a data-file and splits the contents by tabs.
# The first column is an ID, and thus is discarded. The second column consists of the actual reviews data.
# The third column is the true label for each data point.

# The function returns two objects - a list of all reviews, and a numpy array of labels.
# You will need to use this function later.

import math


def get_lists(input_file):
    f=open(input_file, 'r')
    lines = [line.split('\t')[1:] for line in f.readlines()]
    X = [row[0] for row in lines]
    y=np.array([int(row[1]) for row in lines])
    return X, y

# Fill in the following function to take a corpus (list of reviews) as input,
# extract TfIdf values and return an array of features and the vocabulary.

# If the vocabulary argument is supplied, then the function should only convert the input corpus
# to feature vectors using the provided vocabulary and the max_features argument (if not None).
# In this case, the function should return feature vectors and the supplied vocabulary.

# If the max_features parameter is set to None, then all words in the corpus should be used.
# If the max_features parameter is specified (say, k),
# then only use the k most frequent words in the corpus to build your vocabulary.

# The function should return two things.

# The first object should be a numpy array of shape (n_documents, vocab_size),
# which contains the TF-IDF feature vectors for each document.

# The second object should be a dictionary of the words in the vocabulary,
# mapped to their corresponding index in alphabetical sorted order.

def get_tfidf_vectors(token_lists, max_features=None, vocabulary=None):
    def tokenize(text):
        return text.lower().split()

    if vocabulary is None:
        df = defaultdict(int)
        document_tokens = []
        for tokens in token_lists:
            tokens = tokenize(tokens)
            document_tokens.append(tokens)
            unique_tokens = set(tokens)
            for token in unique_tokens:
                df[token] += 1

        if max_features is not None:
            sorted_vocab = sorted(df.items(), key=lambda item: item[1], reverse=True)[:max_features]
            vocabulary = {word: i for i, (word, _) in enumerate(sorted_vocab)}
        else:
            vocabulary = {word: i for i, word in enumerate(sorted(df.keys()))}

        idf = {word: math.log(len(token_lists) / (df[word] + 1)) for word, _ in vocabulary.items()}
    else:
        document_tokens = [tokenize(tokens) for tokens in token_lists]
        idf = {word: math.log(len(token_lists) / (sum(1 for tokens in document_tokens if word in tokens) + 1)) for word in vocabulary}

    tfidf_matrix = np.zeros((len(token_lists), len(vocabulary)))
    for idx, tokens in enumerate(document_tokens):
        tf = Counter(tokens)
        for word, count in tf.items():
            if word in vocabulary:
                tfidf_matrix[idx, vocabulary[word]] = (count / len(tokens)) * idf[word]

    return tfidf_matrix, vocabulary

We will now compare the runtime of our Tf-Idf implementation to the `sklearn` implementation. Call the respective functions with appropriate arguments in the code block below.

In [8]:
# define constants for the files we are using
TRAIN_FILE = "movie_reviews_train.txt"
TEST_FILE = "movie_reviews_test.txt"

train_corpus, y_train = get_lists(TRAIN_FILE)
test_corpus, y_test = get_lists(TEST_FILE)

# First we will use our custom vectorizer to convert words to features, and time it.

start = time.time()
X_train_custom, vocab_custom = get_tfidf_vectors(train_corpus)
end = time.time()
print("Time taken by custom TF-IDF: ", end-start, " seconds")

# Next we will use sklearn's TfidfVectorizer to load in the data, and time it.

sklearn_vectorizer = TfidfVectorizer()
start = time.time()
X_train_sklearn = sklearn_vectorizer.fit_transform(train_corpus)
end = time.time()
print("Time taken by sklearn's TF-IDF: ", end-start, " seconds")

Time taken by custom TF-IDF:  0.3812687397003174  seconds
Time taken by sklearn's TF-IDF:  0.20406103134155273  seconds


NOTE: Ideally, your vectorizer should be within one order of magnitude of the sklearn implementation.

In [9]:
# Any additional code needed to answer questions below.
print('Vocab size of custom vectorizer for train set:', len(vocab_custom))
print('Vocab size of sklearn vectorizer for train set:', len(sklearn_vectorizer.vocabulary_))

non_zero_elements_custom = np.count_nonzero(X_train_custom)
total_elements_custom = X_train_custom.shape[0] * X_train_custom.shape[1]
sparsity_custom = 100 * (1 - (non_zero_elements_custom / total_elements_custom))

sparsity_sklearn = 100 * (1 - (X_train_sklearn.nnz / (X_train_sklearn.shape[0] * X_train_sklearn.shape[1])))
print("Sparsity in Train (custom):", sparsity_custom, "%")
print("Sparsity in Train (sklearn):", sparsity_sklearn, "%")

Vocab size of custom vectorizer for train set: 44077
Vocab size of sklearn vectorizer for train set: 22684
Sparsity in Train (custom): 99.66601770764798 %
Sparsity in Train (sklearn): 99.39492318374185 %


1. How large is the vocabulary generated by your vectorizer?<br> 
- Vocab size of custom vectorizer: 44077
2. How large is the vocabulary generated by the `sklearn` TfidfVectorizer?<br>
- Vocab size of sklearn vectorizer: 22684
3. Where might these differences be coming from?<br> 
- Differences might be coming from different tokenization methods or different handling of case sensitivity.
4. What steps did you take to ensure your vectorizer is optimized for best possible runtime?<br> 
- Used defaultdict for efficient counting.
- Avoided redundant computations by caching results when possible.
5. How sparse are your custom features (average percentage of features per review that are zero)?<br>
- Sparsity (custom): 99.66601770764798 %
6. How sparse are the TfidfVectorizer's features?<br> 
- Sparsity (sklearn): 99.39492318374185 %

NOTE: if you set the lowercase option to False, the sklearn vectorizer should have a vocabulary of around 50k words/tokens.

**Logistic Regression**

Now, we will compare how our custom features stack up against sklearn's TfidfVectorizer, by training two separate Logistic Regression classifiers - one on each set of feature vectors. Then load the test set, and convert it to two sets of feature vectors, one using our custom vectorizer (to do this, provide the vocabulary as a function argument), and one using sklearn's Tfidf (use the same object as before to transform the test inputs). For both classifiers, print the average accuracy on the test set and the F1 score.

In [10]:
from sklearn.metrics import accuracy_score, f1_score

# First use sklearn's LogisticRegression classifier to do sentiment analysis using your custom feature vectors:
X_train_custom, vocab_custom = get_tfidf_vectors(train_corpus)
lr_custom = LogisticRegression()
lr_custom.fit(X_train_custom, y_train)

# Load the test data, extract features using your custom vectorizer, and test the performance of the LR classifier
X_test_custom, _ = get_tfidf_vectors(test_corpus, vocabulary=vocab_custom)
y_pred_custom = lr_custom.predict(X_test_custom)
accuracy_custom = accuracy_score(y_test, y_pred_custom)
f1_custom = f1_score(y_test, y_pred_custom)

# Print the accuracy of your model on the test data
print("Custom TF-IDF Features:")
print("Accuracy:", accuracy_custom)
print("F1 Score:", f1_custom)

# Now repeat the above steps, but this time using features extracted by sklearn's Tfidfvectorizer
sklearn_vectorizer = TfidfVectorizer()
X_train_sklearn = sklearn_vectorizer.fit_transform(train_corpus)
lr_sklearn = LogisticRegression()
lr_sklearn.fit(X_train_sklearn, y_train)

X_test_sklearn = sklearn_vectorizer.transform(test_corpus)
y_pred_sklearn = lr_sklearn.predict(X_test_sklearn)
accuracy_sklearn = accuracy_score(y_test, y_pred_sklearn)
f1_sklearn = f1_score(y_test, y_pred_sklearn)

print("\nSklearn TfidfVectorizer Features:")
print("Accuracy:", accuracy_sklearn)
print("F1 Score:", f1_sklearn)


Custom TF-IDF Features:
Accuracy: 0.8
F1 Score: 0.8290598290598291

Sklearn TfidfVectorizer Features:
Accuracy: 0.775
F1 Score: 0.7906976744186046


NOTE: we're expecting to see a F1 score of around 80% using both your custom features and the sklearn features.

Finally, repeat the process (training and testing), but this time, set the max_features argument to 1000 for both our custom vectorizer and sklearn's Tfidfvectorizer. Report average accuracy and F1 scores for both classifiers.

In [11]:
max_features = 1000
# First use sklearn's LogisticRegression classifier to do sentiment analysis using your custom feature vectors:
X_train_custom, vocab_custom = get_tfidf_vectors(train_corpus, max_features=max_features)
lr_custom = LogisticRegression()
lr_custom.fit(X_train_custom, y_train)


# Load the test data, extract features using your custom vectorizer, and test the performance of the LR classifier
X_test_custom, _ = get_tfidf_vectors(test_corpus, vocabulary=vocab_custom)
y_pred_custom = lr_custom.predict(X_test_custom)
accuracy_custom = accuracy_score(y_test, y_pred_custom)
f1_custom = f1_score(y_test, y_pred_custom)


# Print the accuracy of your model on the test data
print("Custom TF-IDF Features:")
print("Accuracy:", accuracy_custom)
print("F1 Score:", f1_custom)

# Now repeat the above steps, but this time using features extracted by sklearn's Tfidfvectorizer
sklearn_vectorizer = TfidfVectorizer(max_features=max_features)
X_train_sklearn = sklearn_vectorizer.fit_transform(train_corpus)
lr_sklearn = LogisticRegression()
lr_sklearn.fit(X_train_sklearn, y_train)

X_test_sklearn = sklearn_vectorizer.transform(test_corpus)
y_pred_sklearn = lr_sklearn.predict(X_test_sklearn)
accuracy_sklearn = accuracy_score(y_test, y_pred_sklearn)
f1_sklearn = f1_score(y_test, y_pred_sklearn)

print("Sklearn TfidfVectorizer Features:")
print("Accuracy:", accuracy_sklearn)
print("F1 Score:", f1_sklearn)


Custom TF-IDF Features:
Accuracy: 0.78
F1 Score: 0.8135593220338984
Sklearn TfidfVectorizer Features:
Accuracy: 0.78
F1 Score: 0.7981651376146789


1. Is there a stark difference between the two vectorizers with 1000 features?
- No, there isn't a stark difference between the two vectorizers when limited to 1000 features; both show very similar accuracy and F1 scores.
2. Use sklearn's documentation for the Tfidfvectorizer to figure out what may be causing the performance difference (or lack thereof).<br>
- The similar performance is likely because both vectorizers are good at identifying and using the top 1000 most important words for sentiment analysis.

NOTE: Irrespective of your conclusions, both implementations should be above 60% F1 Score.

Task 3: Train a Feedforward Neural Network Model (25 points)
----
1. Using PyTorch, implement a feedforward neural network to do sentiment analysis. This model should take sparse vectors of length 10000 as input (note this is 10000, not 1000), and have a single output with the sigmoid activation function. The number of hidden layers, and intermediate activation choices are up to you, but please make sure your model does not take more than ~1 minute to train.
2. Evaluate the model using PyTorch functions for average accuracy, area under the ROC curve and F1 scores (see [torcheval](https://pytorch.org/torcheval/stable/)) using both vectorizers, with max_features set to 10000 in both cases.

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# if torch.backends.mps.is_available():
# 	device = torch.device("mps")
if torch.cuda.is_available():
	device = torch.device("cuda")
else:
	device = torch.device("cpu")

In [13]:
class feedforward(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(10000, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, X):
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = torch.sigmoid(self.fc3(X))
        return X

    def predict(self, X):
        self.eval()
        with torch.no_grad():
            return self(X)

In [14]:
# Load the data using custom and sklearn vectors
train_corpus, y_train = get_lists(TRAIN_FILE)
test_corpus, y_test = get_lists(TEST_FILE)

vectorizer = TfidfVectorizer(max_features=10000)
X_train_sklearn = vectorizer.fit_transform(train_corpus)
X_test_sklearn = vectorizer.transform(test_corpus)

X_train_custom, vocab_custom = get_tfidf_vectors(train_corpus, max_features=10000)
X_test_custom, _ = get_tfidf_vectors(test_corpus, vocabulary=vocab_custom)

In [15]:
# Train the model for 50 epochs on both custom and sklearn vectors

# Function for training and evaluating the model
def train(X_train, y_train):
    X_train_torch = torch.tensor(X_train).float().to(device)
    y_train_torch = torch.tensor(y_train).float().to(device)

    # Create a DataLoader for the training data
    train_data = TensorDataset(X_train_torch, y_train_torch)
    train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

    # Create a feedforward neural network model
    # you may use any activation function on the hidden layers
    # you should use binary cross-entropy as your loss function
    # Adam is an appropriate optimizer for this task
    model = feedforward().to(device)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters())

    # Train the model
    for epoch in range(50):
        model.train()
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}/50, Loss: {loss.item()}')

    return model

print("Training model with custom features")
custom_model = train(X_train_custom, y_train)

print("\nTraining model with sklearn features")
sklearn_model = train(X_train_sklearn.toarray(), y_train)

Training model with custom features
Epoch 1/50, Loss: 0.6897892951965332
Epoch 2/50, Loss: 0.613271951675415
Epoch 3/50, Loss: 0.44274917244911194
Epoch 4/50, Loss: 0.20252814888954163
Epoch 5/50, Loss: 0.0660819560289383
Epoch 6/50, Loss: 0.04162602871656418
Epoch 7/50, Loss: 0.01783471181988716
Epoch 8/50, Loss: 0.009572441689670086
Epoch 9/50, Loss: 0.0068570408038794994
Epoch 10/50, Loss: 0.0056688860058784485
Epoch 11/50, Loss: 0.0029908674769103527
Epoch 12/50, Loss: 0.0025128794368356466
Epoch 13/50, Loss: 0.0018601553747430444
Epoch 14/50, Loss: 0.0014964134898036718
Epoch 15/50, Loss: 0.0011987502221018076
Epoch 16/50, Loss: 0.0007872257847338915
Epoch 17/50, Loss: 0.0007478757761418819
Epoch 18/50, Loss: 0.0005511886556632817
Epoch 19/50, Loss: 0.0004044831730425358
Epoch 20/50, Loss: 0.0004165777354501188
Epoch 21/50, Loss: 0.0004099368234165013
Epoch 22/50, Loss: 0.00027319847140461206
Epoch 23/50, Loss: 0.0002583259192761034
Epoch 24/50, Loss: 0.00026254975819028914
Epoch 

In [16]:
# !pip install torcheval

# Evaluate the model using custom and sklearn vectors
# Test the model using custom and sklearn vectors
# Evaluate the model and report the score using Binary F1 score, Binary AUROC and Binary accuracy

from torcheval.metrics import BinaryAccuracy, BinaryAUROC, BinaryF1Score

accuracy_metric = BinaryAccuracy()
auroc_metric = BinaryAUROC()
f1_metric = BinaryF1Score()

# Function for evaluating the model
def evaluate_model(model, X_test, y_test):
    X_test_torch = torch.tensor(X_test).float().to(device)
    y_test_torch = torch.tensor(y_test).float().to(device)
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_torch).squeeze()

    # Update metric calculations
    accuracy_metric.update(outputs, y_test_torch)
    auroc_metric.update(outputs, y_test_torch)
    f1_metric.update(outputs, y_test_torch)

    # Compute final metrics
    final_accuracy = accuracy_metric.compute()
    final_auroc = auroc_metric.compute()
    final_f1 = f1_metric.compute()

    accuracy_metric.reset()
    auroc_metric.reset()
    f1_metric.reset()

    return final_accuracy.item(), final_auroc.item(), final_f1.item()

custom_accuracy, custom_auroc, custom_f1 = evaluate_model(custom_model, X_test_custom, y_test)
sklearn_accuracy, sklearn_auroc, sklearn_f1 = evaluate_model(sklearn_model, X_test_sklearn.toarray(), y_test)

print("Custom TF-IDF Model:")
print("Accuracy:", custom_accuracy)
print("AUROC:", custom_auroc)
print("F1 Score:", custom_f1)

print("\nSklearn TfidfVectorizer Model:")
print("Accuracy:", sklearn_accuracy)
print("AUROC:", sklearn_auroc)
print("F1 Score:", sklearn_f1)

Custom TF-IDF Model:
Accuracy: 0.7850000262260437
AUROC: 0.8937392882347011
F1 Score: 0.7962085604667664

Sklearn TfidfVectorizer Model:
Accuracy: 0.8100000023841858
AUROC: 0.8970662365157778
F1 Score: 0.8155339360237122


NOTE: As in the last task, we're expecting to see a F1 score of over 60% using both your custom features and the sklearn features.

5 points in this assignment are reserved for overall style (both for writing and for code submitted). All work submitted should be clear, easily interpretable, and checked for spelling, etc. (Re-read what you write and make sure it makes sense). Course staff are always happy to give grammatical help (but we won't pre-grade the content of your answers).